# Data Profiler - What's in your data?

This introductory jupyter notebook demonstrates the basic usages of the Data Profiler. The library is designed to easily detect sensitive data and gather statistics on your datasets with just several lines of code. The Data Profiler can handle several different data types including: CSV (or any delimited file), JSON, Parquet, AVRO, and text. Additionally, there are a plethora of options to customize your profile. This library also has the ability to update profiles from multiple batches of large datasets, or merge multiple profiles. In particular, this example covers the followings:

    - Basic usage of the Data Profiler
    - The data reader class
    - Profiler options
    - Updating profiles and merging profiles

First, let's import the libraries needed for this example.

In [16]:
import os
import sys
import json
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, '..')
import dataprofiler as dp

data_path = "../dataprofiler/tests/data"

## Basic Usage of the Data Profiler

This section shows the basic example of the Data Profiler. A CSV dataset is read using the data reader, then the Data object is given to the Data Profiler to detect sensitive data and obtain the statistics.

In [17]:
# use data reader to read input data
data = dp.Data(os.path.join(data_path, "csv/aws_honeypot_marx_geo.csv"))
print(data.data.head())

# run data profiler and get the report
profile = dp.Profiler(data)
report  = profile.report(report_options={"output_format":"compact"})

# print the report
print(json.dumps(report, indent=4))

       datetime               host         src proto type srcport destport  \
0  3/3/13 21:53     groucho-oregon  1032051418   TCP         6000     1433   
1  3/3/13 21:57     groucho-oregon  1347834426   UDP         5270     5060   
2  3/3/13 21:58     groucho-oregon  2947856490   TCP         2489     1080   
3  3/3/13 21:58                                  UDP        43235     1900   
4  3/3/13 21:58  groucho-singapore  3587648279   TCP        56577       80   

             srcip         locale localeabbr postalcode latitude  longitude  \
0   61.131.218.218  Jiangxi Sheng         36               28.55   115.9333   
1      80.86.82.58                                            51          9   
2  175.180.184.106         Taipei                        25.0392    121.525   
3                          Oregon         OR      97124  45.5848  -122.9117   
4    213.215.43.23                                         48.86       2.35   

  owner                                            comme

  0%|          | 0/16 [00:00<?, ?it/s]

Finding the Null values in the columns... (with 15 processes)


  0%|          | 0/16 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


...

 62%|██████▎   | 10/16 [00:02<00:00,  7.79it/s]

 69%|██████▉   | 11/16 [00:02<00:00,  7.91it/s]

 81%|████████▏ | 13/16 [00:02<00:00,  7.79it/s]

100%|██████████| 16/16 [00:02<00:00,  5.63it/s]


{
    "global_stats": {
        "samples_used": 2999,
        "column_count": 16,
        "row_count": 2999,
        "row_has_null_ratio": 1.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "csv",
        "encoding": "utf-8"
    },
    "data_stats": {
        "datetime": {
            "column_name": "datetime",
            "data_type": "datetime",
            "data_label": "DATETIME|FLOAT",
            "categorical": false,
            "order": "random",
            "samples": "['3/4/13 4:18', '3/4/13 23:08', '3/5/13 7:00', '3/5/13 0:14',\n '3/4/13 13:44']",
            "statistics": {
                "min": "3/3/13 21:53",
                "max": "3/25/13 16:34",
                "format": "['%m/%d/%y %H:%M']",
                "unique_count": 1400,
                "unique_ratio": 0.4671,
                "sample_size": 2999,
                "null_count": 2,
                "null_types": "['']",
                "d

The report includes `global_stats` and `data_stats` for the given dataset. The former contains overall properties of the data such as number of rows/columns, null ratio, duplicate ratio, while the latter contains specific properties and statistics for each column such as detected data label, min, max, mean, variance, etc. In this example, the `compact` format of the report is used to shorten the full list of the results. To get more results related to detailed predictions at the entity level from the Data Labeler component or histogram results, the format `pretty` should be used.

## Data reader class

DataProfiler can detect multiple file types including CSV (or any delimited file), JSON, Parquet, AVRO, and text. The example below shows that it successfully detects data types from multiple categories regardless of the file extensions.

In [18]:
# use data reader to read input data with different file types
csv_files = [
    "csv/aws_honeypot_marx_geo.csv",
    "csv/all-strings-skip-header-author.csv", # csv files with the author/description on the first line
    "csv/sparse-first-and-last-column-empty-first-row.txt", # csv file with the .txt extension
]
json_files = [
    "json/complex_nested.json",
    "json/honeypot_intentially_mislabeled_file.csv", # json file with the .csv extension
]
parquet_files = [
    "parquet/nation.dict.parquet",
    "parquet/nation.plain.intentionally_mislabled_file.csv", # parquet file with the .csv extension
]
avro_files = [
    "avro/userdata1.avro",
    "avro/userdata1_intentionally_mislabled_file.json", # avro file with the .json extension
]
text_files = [
    "txt/discussion_reddit.txt",
]

all_files = {
    "csv": csv_files,
    "json": json_files,
    "parquet": parquet_files,
    "avro": avro_files,
    "text": text_files
}

for file_type in all_files:
    print(file_type)
    for file in all_files[file_type]:
        data = dp.Data(os.path.join(data_path, file))
        print("{:<85} {:<15}".format(file, data.data_type))
    print("\n")

csv
csv/aws_honeypot_marx_geo.csv                                                         csv            
csv/all-strings-skip-header-author.csv                                                csv            
csv/sparse-first-and-last-column-empty-first-row.txt                                  csv            


json
json/complex_nested.json                                                              json           
json/honeypot_intentially_mislabeled_file.csv                                         json           


parquet
parquet/nation.dict.parquet                                                           parquet        
parquet/nation.plain.intentionally_mislabled_file.csv                                 parquet        


avro
Install charset_normalizer for improved file encoding detection
Install charset_normalizer for improved file encoding detection
avro/userdata1.avro                                                                   avro           
Install charset_normalizer f

The `Data` class detects the file type and uses one of the following classes: `CSVData`, `JSONData`, `ParquetData`, `AVROData`, `TextData`. Users can call these specific classes directly if desired. For example, below we provide a collection of data with different types, each of them is processed by the corresponding data class.

In [19]:
# use individual data reader classes
from dataprofiler.data_readers.csv_data import CSVData
from dataprofiler.data_readers.json_data import JSONData
from dataprofiler.data_readers.parquet_data import ParquetData
from dataprofiler.data_readers.avro_data import AVROData
from dataprofiler.data_readers.text_data import TextData

csv_files = "csv/aws_honeypot_marx_geo.csv"
json_files = "json/complex_nested.json"
parquet_files = "parquet/nation.dict.parquet"
avro_files = "avro/userdata1.avro"
text_files = "txt/discussion_reddit.txt"

all_files = {
    "csv": [csv_files, CSVData],
    "json": [json_files, JSONData],
    "parquet": [parquet_files, ParquetData],
    "avro": [avro_files, AVROData],
    "text": [text_files, TextData],
}

for file_type in all_files:
    file, data_reader = all_files[file_type]
    data = data_reader(os.path.join(data_path, file))
    print("File name {}\n".format(file))
    if file_type == "text":
        print(data.data[0][:1000]) # print the first 1000 characters
    else:
        print(data.data)
    print('===============================================================================')

File name csv/aws_honeypot_marx_geo.csv

           datetime               host         src proto type srcport  \
0      3/3/13 21:53     groucho-oregon  1032051418   TCP         6000   
1      3/3/13 21:57     groucho-oregon  1347834426   UDP         5270   
2      3/3/13 21:58     groucho-oregon  2947856490   TCP         2489   
3      3/3/13 21:58                                  UDP        43235   
4      3/3/13 21:58  groucho-singapore  3587648279   TCP        56577   
...             ...                ...         ...   ...  ...     ...   
2994   3/5/13 16:34     groucho-norcal  1604972691   TCP        10699   
2995   3/5/13 16:34       zeppo-norcal  1604972691   TCP        27067   
2996   3/5/13 16:34     groucho-norcal  1604972691   TCP        10699   
2997   3/5/13 16:34       zeppo-norcal  1604972691   TCP        27067   
2998  3/25/13 16:34     groucho-norcal  1604972691   TCP        10699   

     destport            srcip         locale localeabbr postalcode latitude  \
0 

In addition to reading the input data from multiple file types, the Data Profiler allows the input data as a dataframe.

In [20]:
# run data profiler and get the report
my_dataframe = pd.DataFrame([[1, 2.0],[1, 2.2],[-1, 3]], columns=["col_int", "col_float"])
profile = dp.Profiler(my_dataframe)
report  = profile.report(report_options={"output_format":"compact"})

# Print the report
print(json.dumps(report, indent=4))

100%|██████████| 2/2 [00:00<00:00, 768.12it/s]

Finding the Null values in the columns...



  0%|          | 0/2 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


100%|██████████| 2/2 [00:00<00:00,  2.25it/s]

{
    "global_stats": {
        "samples_used": 3,
        "column_count": 2,
        "row_count": 3,
        "row_has_null_ratio": 0.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "<class 'pandas.core.frame.DataFrame'>",
        "encoding": null
    },
    "data_stats": {
        "col_int": {
            "column_name": "col_int",
            "data_type": "int",
            "data_label": "ORDINAL",
            "categorical": true,
            "order": "descending",
            "samples": "['-1', '1', '1']",
            "statistics": {
                "min": -1.0,
                "max": 1.0,
                "mean": 0.3333,
                "variance": 1.3333,
                "stddev": 1.1547,
                "quantiles": {
                    "0": -0.9985,
                    "1": 0.9985,
                    "2": 0.9992
                },
                "unique_count": 2,
                "unique_ratio": 0.6667

## Profiler options

The Data Profiler can enable/disable statistics and modify features through profiler options. For example, if the users only want the statistics information, they may turn off the Data Labeler functionality. Below, let's remove the histogram and data labeler component while running Data Profiler.

In [21]:
profile_options = dp.ProfilerOptions()
profile_options.set({"histogram_and_quantiles.is_enabled": False,
                     "data_labeler.is_enabled": False,})

profile = dp.Profiler(my_dataframe, profiler_options=profile_options)
report  = profile.report(report_options={"output_format":"pretty"})

# Print the report
print(json.dumps(report, indent=4))

  0%|          | 0/2 [00:00<?, ?it/s]

Finding the Null values in the columns...


  0%|          | 0/2 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


100%|██████████| 2/2 [00:00<00:00,  2.56it/s]

{
    "global_stats": {
        "samples_used": 3,
        "column_count": 2,
        "row_count": 3,
        "row_has_null_ratio": 0.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "<class 'pandas.core.frame.DataFrame'>",
        "encoding": null
    },
    "data_stats": {
        "col_int": {
            "column_name": "col_int",
            "data_type": "int",
            "categorical": true,
            "order": "descending",
            "samples": "['1', '1', '-1']",
            "statistics": {
                "min": -1.0,
                "max": 1.0,
                "mean": 0.3333,
                "variance": 1.3333,
                "stddev": 1.1547,
                "histogram": {
                    "bin_counts": null,
                    "bin_edges": null
                },
                "quantiles": {
                    "0": null,
                    "1": null,
                    "2": null
        

Besides toggling on and off features, other options like the data labeler sample size or histogram bin method can be directly set and validated as shown here:

In [22]:
profile_options = dp.ProfilerOptions()
profile_options.structured_options.data_labeler.sample_size = 1
profile_options.structured_options.int.histogram_and_quantiles.bin_count_or_method = "rice"
# An error will raise if the options are set incorrectly.
profile_options.validate()

profile = dp.Profiler(my_dataframe, profiler_options=profile_options)
report  = profile.report(report_options={"output_format":"pretty"})

# Print the report
print(json.dumps(report, indent=4))

100%|██████████| 2/2 [00:00<00:00, 791.08it/s]

Finding the Null values in the columns...



  0%|          | 0/2 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


100%|██████████| 2/2 [00:00<00:00,  2.13it/s]

{
    "global_stats": {
        "samples_used": 3,
        "column_count": 2,
        "row_count": 3,
        "row_has_null_ratio": 0.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "<class 'pandas.core.frame.DataFrame'>",
        "encoding": null
    },
    "data_stats": {
        "col_int": {
            "column_name": "col_int",
            "data_type": "int",
            "data_label": "ORDINAL",
            "categorical": true,
            "order": "descending",
            "samples": "['1', '1', '-1']",
            "statistics": {
                "min": -1.0,
                "max": 1.0,
                "mean": 0.3333,
                "variance": 1.3333,
                "stddev": 1.1547,
                "histogram": {
                    "bin_edges": "[-1.        , -0.33333333,  0.33333333,  1.        ]",
                    "bin_counts": "[1., 0., 2.]"
                },
                "quantiles": {
   

## Update profiles

One of the interesting features of the Data Profiler is the ability to update profiles from batches of data, which allows for data streaming usage. In this section, the original dataset is separated into two batches with equal size. Each batch is then updated with Data Profiler sequentially.  

After the update, we expect the resulted profiles give the same statistics as the profiles updated from the full dataset. We will verify that through some properties in `global_stats` of the profiles including `column_count`, `row_count`, `row_is_null_ratio`, `duplicate_row_count`. 

In [33]:
# read the input data and devide it into two equal halves
data = dp.Data(os.path.join(data_path, "csv/aws_honeypot_marx_geo.csv"))
df = data.data
df1 = df.iloc[:int(len(df)/2)]
df2 = df.iloc[int(len(df)/2):]

# Update the profile with the first half
profile = dp.Profiler(df1)

# Update the profile with the second half
profile.update_profile(df2)

# Update profile with the full dataset
profile_full = dp.Profiler(df)

report  = profile.report(report_options={"output_format":"compact"})
report_full  = profile_full.report(report_options={"output_format":"compact"})

# print the report
print(json.dumps(report, indent=4))
print(json.dumps(report_full, indent=4))

  0%|          | 0/16 [00:00<?, ?it/s]

Finding the Null values in the columns... (with 15 processes)


  0%|          | 0/16 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


  6%|▋         | 1/16 [00:00<00:13,  1.08it/s]

...

 62%|██████▎   | 10/16 [00:02<00:00,  7.70it/s]

 69%|██████▉   | 11/16 [00:02<00:00,  7.70it/s]

 81%|████████▏ | 13/16 [00:02<00:00,  7.66it/s]

100%|██████████| 16/16 [00:02<00:00,  5.55it/s]


{
    "global_stats": {
        "samples_used": 2999,
        "column_count": 16,
        "row_count": 2999,
        "row_has_null_ratio": 1.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "<class 'pandas.core.frame.DataFrame'>",
        "encoding": null
    },
    "data_stats": {
        "datetime": {
            "column_name": "datetime",
            "data_type": "datetime",
            "data_label": "DATETIME|FLOAT",
            "categorical": false,
            "order": "random",
            "samples": "['3/5/13 2:04', '3/5/13 0:10', '3/5/13 12:11', '3/5/13 10:07',\n '3/4/13 22:28']",
            "statistics": {
                "min": "3/3/13 21:53",
                "max": "3/25/13 16:34",
                "histogram": null,
                "format": "['%m/%d/%y %H:%M']",
                "times": {
                    "datetime": 0.1103,
                    "data_labeler_predict": 0.2152
                },


You can see that the profiles are exactly the same whether they are broken into several updates or not.

## Merge profiles

In addition to the profile update, Data Profiler provides the merging functionality which allows users to combine the profiles updated from multiple locations. This enables Data Profiler to be used in a distributed computing environment. Below, we assume that the two aforementioned halves of the original dataset come from two different machines. Each of them is then updated with the Data Profiler on the same machine, then the resulted profiles are merged.

As with the profile update, we expect the merged profiles give the same statistics as the profiles updated from the full dataset.

In [35]:
# Update the profile with the first half
profile1 = dp.Profiler(df1)

# Update the profile with the second half
profile2 = dp.Profiler(df2)

# merge profiles
profile_merge = profile1 + profile2

# check results of the merged profile
report_merge  = profile.report(report_options={"output_format":"compact"})

# print the report
print(json.dumps(report_merge, indent=4))
print(json.dumps(report_full, indent=4))

  0%|          | 0/16 [00:00<?, ?it/s]

Finding the Null values in the columns... (with 15 processes)


  0%|          | 0/16 [00:00<?, ?it/s]

Calculating the statistics...  (with 4 processes)


  6%|▋         | 1/16 [00:00<00:13,  1.10it/s]

...

  6%|▋         | 1/16 [00:01<00:15,  1.06s/it]

 19%|█▉        | 3/16 [00:01<00:04,  2.77it/s]

 25%|██▌       | 4/16 [00:01<00:03,  3.58it/s]

100%|██████████| 16/16 [00:02<00:00,  5.94it/s]


{
    "global_stats": {
        "samples_used": 2999,
        "column_count": 16,
        "row_count": 2999,
        "row_has_null_ratio": 1.0,
        "row_is_null_ratio": 0.0,
        "unique_row_ratio": 1.0,
        "duplicate_row_count": 0,
        "file_type": "<class 'pandas.core.frame.DataFrame'>",
        "encoding": null
    },
    "data_stats": {
        "datetime": {
            "column_name": "datetime",
            "data_type": "datetime",
            "data_label": "DATETIME|FLOAT",
            "categorical": false,
            "order": "random",
            "samples": "['3/5/13 2:04', '3/5/13 0:10', '3/5/13 12:11', '3/5/13 10:07',\n '3/4/13 22:28']",
            "statistics": {
                "min": "3/3/13 21:53",
                "max": "3/25/13 16:34",
                "format": "['%m/%d/%y %H:%M']",
                "unique_count": 1400,
                "unique_ratio": 0.4671,
                "sample_size": 2999,
                "null_count": 2,
                "null_ty

You can see that the profiles are exactly the same!

## Conclusion

We have walked through some basic examples of Data Profiler usage, with different input data types and profiling options. We also work with update and merging functionality of the Data Profiler, which make it applicable for data streaming and distributed environment. Interested users can try with different datasets and functionalities as desired.